以单元测试模拟preds / batch 的形状来跑通一些逻辑，这个ipynb文件用于测试`loss.py`，其位于`ultralytics/yolo/utils/loss.py`

# 测试 `v8DetectionLoss`

In [3]:
# 简单单测脚本：模拟 feats / batch 并运行 v8DetectionLoss 的前向（CPU）
# 运行方式（在 repo 根目录）：python -m tests.test_loss_forward
import sys
from pathlib import Path
repo_root = Path("/Users/idealistsiren/repos/yolov8_segment_pose").resolve()
print(repo_root)
sys.path.insert(0, str(repo_root))

import torch
import torch.nn as nn
from ultralytics.yolo.utils.loss import v8DetectionLoss

# 构造一个最小的 fake model，满足 v8DetectionLoss 在 __init__ 中的需求
class FakeHead:
    def __init__(self, stride, nc, reg_max):
        # stride tensor: e.g. [8,16,32]
        self.stride = torch.tensor(stride)
        self.nc = nc
        self.reg_max = reg_max
        # no = reg_max * 4 + nc (与 parse_model 中保持一致)
        self.no = reg_max * 4 + nc

class DummyModel(nn.Module):
    def __init__(self, head):
        super().__init__()
        # 需要至少一个 parameter 使 next(model.parameters()) 有值以获取 device
        self.dummy = nn.Parameter(torch.zeros(1))
        # model attribute: list-like with last element having stride/nc/no/reg_max
        self.model = [head]
        # args: 为损失使用提供超参字典（可按实际训练超参调整）
        class H: pass
        h = H()
        # 典型超参名称：box, cls, dfl
        h.box = 0.05
        h.cls = 0.5
        h.dfl = 0.0
        # 其他训练中可能存在的键（安全起见）
        h.pose = 1.0
        h.kobj = 1.0
        self.args = h

def make_fake_feats(B, no, sizes):
    # sizes: list of (H,W) for each feature map
    feats = []
    for H, W in sizes:
        # create random tensor (B, no, H, W)
        feats.append(torch.randn(B, no, H, W))
    return feats

def make_batch(batch_size):
    # 构造 1 gt per image 的简单批次（normalized xywh）
    # batch['batch_idx']: (num_targets,)
    # batch['cls']: (num_targets,)
    # batch['bboxes']: (num_targets, 4) (xywh normalized [0,1])
    batch_idx = torch.arange(batch_size)
    cls = torch.zeros(batch_size, dtype=torch.long)
    # place gt near center with moderate size
    bboxes = torch.stack([
        torch.tensor([0.5, 0.5, 0.2, 0.2], dtype=torch.float),
    ] * batch_size, dim=0)
    return {
        'batch_idx': batch_idx,
        'cls': cls,
        'bboxes': bboxes
    }

def main():
    device = torch.device('cpu')
    stride = [8.0, 16.0, 32.0]
    nc = 3
    reg_max = 8  # >1 表示启用 DFL 支持路径（代码会处理）
    head = FakeHead(stride=stride, nc=nc, reg_max=reg_max)
    model = DummyModel(head).to(device)

    loss_fn = v8DetectionLoss(model)

    B = 2
    # choose feature sizes that match typical downsampling: (H = img_h/stride)
    # 假设 imgsz = stride[0]*H0，取 H0=80/8 等，任意小尺寸也可
    sizes = [(20, 20), (10, 10), (5, 5)]  # 三尺度示例
    feats = make_fake_feats(B, head.no, sizes)

    batch = make_batch(B)

    # 运行 loss 前向
    try:
        loss, loss_items = loss_fn(feats, batch)
        print("LOSS:", loss.item())
        print("LOSS items:", loss_items)
    except Exception as e:
        print("Error running loss forward:", e)
        raise

if __name__ == "__main__":
    main()

/Users/idealistsiren/repos/yolov8_segment_pose


/Users/idealistsiren/miniconda3/envs/yolov8_seg_pose/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/idealistsiren/repos/yolov8_segment_pose/ultralytics/yolo/utils/checks.py:16: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg


LOSS: 1203.2271728515625
LOSS items: tensor([2.9056e-02, 6.0158e+02, 0.0000e+00])
